### Due to GitHub's file size limit (maximum 100MB per file), the larger RoBERTa-based models are not included in this repository.

#### Specifically, files under ./models/* and ./packages/* are excluded from version control.

### 由于 GitHub 对单个文件大小的限制（最大 100MB），基于 RoBERTa 的大规模模型未包含在此仓库中。

##### 具体而言，./models/* 与 ./packages/* 路径下的文件未纳入版本控制。

In [1]:
# =========================
# Cell 0：基础导入 & 路径设置
# =========================
from pathlib import Path
import os
import spacy
from spacy.lookups import Lookups
from spacy.tokens import DocBin
from spacy.cli.package import package

# 项目根目录
project_root = Path(".").resolve()

# 目录结构
corpus_dir = project_root / "corpus"
models_dir = project_root / "models"
model_name = "lv_roberta_large"
trained_model_path = models_dir / model_name / "model-best"
final_model_path = models_dir / model_name / "model_roberta_large"
lookups_path = project_root / "lookups_lv"
package_output_dir = project_root / "packages"
config_path = project_root / "config" / "config_roberta_large.cfg"

# 创建目录
for p in [corpus_dir, models_dir, models_dir / model_name, package_output_dir, lookups_path, project_root / "config"]:
    p.mkdir(parents=True, exist_ok=True)

print("✅ 目录结构准备完成")



✅ 目录结构准备完成


In [2]:
# ===============================
# Cell 1：Conllu 转换为 spaCy 格式
# ===============================
!python -m spacy convert ud_latvian/lv_lvtb-ud-train.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-dev.conllu ./corpus -n 10
!python -m spacy convert ud_latvian/lv_lvtb-ud-test.conllu ./corpus -n 10


ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1506 documents):
corpus/lv_lvtb-ud-train.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (208 documents):
corpus/lv_lvtb-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (240 documents):
corpus/lv_lvtb-ud-test.spacy


In [3]:
# =============================
# Cell 2：初始化 config
# =============================
!python -m spacy init config ./config/config_roberta_large.cfg \
    --lang lv \
    --pipeline transformer,tagger,morphologizer,parser,senter \
    --optimize efficiency \
    --gpu


ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser, senter
- Optimize for: efficiency
- Hardware: GPU
- Transformer: bert-base-multilingual-uncased
✔ Auto-filled config with all values
✔ Saved config
config/config_roberta_large.cfg
You can now add your data and train your pipeline:
python -m spacy train config_roberta_large.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [4]:
# ==========================
# Cell 3：修改 config
# ==========================

cfg_text = config_path.read_text(encoding="utf-8")

# 替换训练/验证集路径
cfg_text = cfg_text.replace("train = null", f"train = {corpus_dir}/lv_lvtb-ud-train.spacy")
cfg_text = cfg_text.replace("dev = null", f"dev = {corpus_dir}/lv_lvtb-ud-dev.spacy")

# 替换 transformer 模型
cfg_text = cfg_text.replace("bert-base-multilingual-uncased", "xlm-roberta-large")

# 开启混合精度
cfg_text = cfg_text.replace("mixed_precision = false", "mixed_precision = true")

config_path.write_text(cfg_text, encoding="utf-8")
print("✅ config 已更新训练/验证路径、transformer 模型，并启用混合精度")


✅ config 已更新训练/验证路径、transformer 模型，并启用混合精度


In [5]:
# ================================
# Cell 4：生成 lemma lookup table
# ================================

# 加载训练集
docbin = DocBin().from_disk(corpus_dir / "lv_lvtb-ud-train.spacy")
lemma_dict = {}

for doc in docbin.get_docs(spacy.blank("lv").vocab):
    for token in doc:
        if token.lemma_:
            lemma_dict[token.text.lower()] = token.lemma_

lookups = Lookups()
lookups.add_table("lemma_lookup", lemma_dict)
lookups.to_disk(lookups_path)
print(f"✅ 已生成 lemma lookup table，路径: {lookups_path}")


/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ 已生成 lemma lookup table，路径: /home/jesse/Projects/myprojs/spaCy_Pipeline/LV_RoBERTa_large/lookups_lv


In [6]:
# =============================
# Cell 5：训练 RoBERTa_large 模型
# =============================
!python -m spacy train ./config/config_roberta_large.cfg \
    --output ./models/lv_roberta_large\
    --paths.train ./corpus/lv_lvtb-ud-train.spacy \
    --paths.dev ./corpus/lv_lvtb-ud-dev.spacy \
    --gpu-id 0


ℹ Saving to output directory: models/lv_roberta_large
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100%|██████████████████| 25.0/25.0 [00:00<00:00, 402kB/s]
config.json: 100%|█████████████████████████████| 616/616 [00:00<00:00, 10.9MB/s]
sentencepiece.bpe.model: 100%|█████████████| 5.07M/5.07M [00:01<00:00, 2.98MB/s]
tokenizer.json: 100%|██████████████████████| 9.10M/9.10M [00:01<00:00, 5.80MB/s]
model.safetensors: 100%|███████████████████| 2.24G/2.24G [02:43<00:00, 13.7MB/s]
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'tagger', 'morphologizer', 'parser',
'senter']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TAGGER  LOSS MORPH...  LOSS PARSER  LOSS SENTER  TAG_ACC  POS_ACC  MORPH_ACC  DEP_UAS  DEP_LAS  SENTS_P  SENTS_R  SENTS_F  SCORE 
---  ------  -------------  -----------  -------------  -----------  ----------- 

In [7]:
# ========================
# Cell 6: 模型评估
# ========================
!python -m spacy evaluate ./models/lv_roberta_large//model-best ./corpus/lv_lvtb-ud-test.spacy --gpu-id 0

ℹ Using GPU: 0

================================== Results ==================================

TOK      99.53
TAG      93.35
POS      98.11
MORPH    96.32
UAS      92.53
LAS      89.31
SENT P   97.09
SENT R   97.41
SENT F   97.25
SPEED    13939


============================== MORPH (per feat) ==============================

                P        R        F
ExtPos      93.94    89.21    91.51
Case        98.45    97.65    98.05
Gender      97.76    96.80    97.28
Number      98.44    97.54    97.99
Person      99.05    98.97    99.01
PronType    98.74    98.46    98.60
Evident     99.03    99.25    99.14
Mood        98.77    98.72    98.74
Polarity    99.46    99.61    99.53
Tense       97.69    98.07    97.88
VerbForm    99.08    99.24    99.16
Voice       99.22    99.36    99.29
Definite    98.24    98.02    98.13
Degree      99.16    98.75    98.95
Poss       100.00   100.00   100.00
NumType     99.61    76.91    86.80
Reflex      99.14    99.46    99.30
Aspect      98.36    99.0

In [8]:
# ==========================================
# Cell 7：添加 Lemmatizer (lookup) 并保存模型
# ==========================================
import spacy
from spacy.lookups import Lookups
from pathlib import Path

trained_model_path = "./models/lv_roberta_large/model-best"
final_model_path = "./models/lv_roberta_large/model_roberta_large"
lookups_path = "./lookups_lv"

# 1️⃣ 加载训练好的模型
nlp = spacy.load(trained_model_path)

# 2️⃣ 加载 lookups
lookups = Lookups().from_disk(lookups_path)

# 3️⃣ 添加 lemmatizer（lookup 模式）并直接传入 lookups
lemmatizer = nlp.add_pipe("lemmatizer", config={"mode":"lookup"}, last=True)
lemmatizer.lookups = lookups  # ⚡ 注意这里直接赋值属性，而不是在 config 里传

# 4️⃣ 保存最终模型
nlp.to_disk(final_model_path)
print(f"✅ 模型已保存到 {final_model_path}，包含 lemmatizer + lookups")


✅ 模型已保存到 ./models/lv_roberta_large/model_roberta_large，包含 lemmatizer + lookups


In [9]:
# =======================
# Cell 8：打包模型
# =======================

from spacy.cli.package import package
from pathlib import Path
import os

project_root = Path(".").resolve()
os.environ["PYTHONPATH"] = f"{project_root}:{os.environ.get('PYTHONPATH','')}"

# ⚡ 注意这里把字符串路径换成 Path 对象
package(
    input_dir=Path(final_model_path),
    output_dir=Path(package_output_dir),
    name="roberta_large",
    version="1.0.0",
    force=True
)

print(f"✅ 打包完成，发布包在 {package_output_dir} 文件夹里")

* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info
creating lv_roberta_large.egg-info
writing lv_roberta_large.egg-info/PKG-INFO
writing dependency_links to lv_roberta_large.egg-info/dependency_links.txt
writing entry points to lv_roberta_large.egg-info/entry_points.txt
writing requirements to lv_roberta_large.egg-info/requires.txt
writing top-level names to lv_roberta_large.egg-info/top_level.txt
writing manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
reading manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
* Building sdist...


running sdist
running egg_info
writing lv_roberta_large.egg-info/PKG-INFO
writing dependency_links to lv_roberta_large.egg-info/dependency_links.txt
writing entry points to lv_roberta_large.egg-info/entry_points.txt
writing requirements to lv_roberta_large.egg-info/requires.txt
writing top-level names to lv_roberta_large.egg-info/top_level.txt
reading manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
running check
creating lv_roberta_large-1.0.0
creating lv_roberta_large-1.0.0/lv_roberta_large
creating lv_roberta_large-1.0.0/lv_roberta_large.egg-info
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0/lemmatizer/lookups
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0/morphologizer
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0/parser
creating 

In [10]:
# =======================
# Cell 9：生成 wheel + sdist
# =======================
import subprocess
from pathlib import Path

package_output_dir = Path("./packages/lv_roberta_large-1.0.0")

# 进入打包目录生成 wheel 和 sdist
subprocess.run(
    ["python", "-m", "build", "--wheel", "--sdist"],
    cwd=str(package_output_dir)
)

print(f"✅ wheel + sdist 已生成，路径: {package_output_dir / 'dist'}")


* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for wheel...
running egg_info
writing lv_roberta_large.egg-info/PKG-INFO
writing dependency_links to lv_roberta_large.egg-info/dependency_links.txt
writing entry points to lv_roberta_large.egg-info/entry_points.txt
writing requirements to lv_roberta_large.egg-info/requires.txt
writing top-level names to lv_roberta_large.egg-info/top_level.txt
reading manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
* Building wheel...


running bdist_wheel
running build
running build_py
creating build/lib/lv_roberta_large
copying lv_roberta_large/__init__.py -> build/lib/lv_roberta_large
creating build/lib/lv_roberta_large/lv_roberta_large-1.0.0
copying lv_roberta_large/lv_roberta_large-1.0.0/meta.json -> build/lib/lv_roberta_large/lv_roberta_large-1.0.0
copying lv_roberta_large/lv_roberta_large-1.0.0/README.md -> build/lib/lv_roberta_large/lv_roberta_large-1.0.0
copying lv_roberta_large/lv_roberta_large-1.0.0/tokenizer -> build/lib/lv_roberta_large/lv_roberta_large-1.0.0
copying lv_roberta_large/lv_roberta_large-1.0.0/config.cfg -> build/lib/lv_roberta_large/lv_roberta_large-1.0.0
creating build/lib/lv_roberta_large/lv_roberta_large-1.0.0/morphologizer
copying lv_roberta_large/lv_roberta_large-1.0.0/morphologizer/cfg -> build/lib/lv_roberta_large/lv_roberta_large-1.0.0/morphologizer
copying lv_roberta_large/lv_roberta_large-1.0.0/morphologizer/model -> build/lib/lv_roberta_large/lv_roberta_large-1.0.0/morphologizer
c

adding 'lv_roberta_large/lv_roberta_large-1.0.0/tagger/model'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/transformer/cfg'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/transformer/model'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/vocab/key2row'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/vocab/lookups.bin'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/vocab/strings.json'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/vocab/vectors'
adding 'lv_roberta_large/lv_roberta_large-1.0.0/vocab/vectors.cfg'
adding 'lv_roberta_large-1.0.0.dist-info/METADATA'
adding 'lv_roberta_large-1.0.0.dist-info/WHEEL'
adding 'lv_roberta_large-1.0.0.dist-info/entry_points.txt'
adding 'lv_roberta_large-1.0.0.dist-info/top_level.txt'
adding 'lv_roberta_large-1.0.0.dist-info/RECORD'
removing build/bdist.linux-x86_64/wheel
* Creating isolated environment: venv+pip...
* Installing packages in isolated environment:
  - setuptools >= 40.8.0
* Getting build dependencies for sdist...
running egg_info

running sdist
running egg_info
writing lv_roberta_large.egg-info/PKG-INFO
writing dependency_links to lv_roberta_large.egg-info/dependency_links.txt
writing entry points to lv_roberta_large.egg-info/entry_points.txt
writing requirements to lv_roberta_large.egg-info/requires.txt
writing top-level names to lv_roberta_large.egg-info/top_level.txt
reading manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'lv_roberta_large.egg-info/SOURCES.txt'
running check
creating lv_roberta_large-1.0.0
creating lv_roberta_large-1.0.0/lv_roberta_large
creating lv_roberta_large-1.0.0/lv_roberta_large.egg-info
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0/lemmatizer/lookups
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0/morphologizer
creating lv_roberta_large-1.0.0/lv_roberta_large/lv_roberta_large-1.0.0/parser
creating 

In [11]:
# ==========================
# Cell 10A: 通过wheel文件安装
# ==========================
import subprocess
import spacy
import pandas as pd


# ⚡ 安装 wheel
subprocess.run([
    "pip", 
    "install", 
    "./packages/lv_roberta_large-1.0.0/dist/lv_roberta_large-1.0.0-py3-none-any.whl"
])

# 加载模型
nlp_xlmr = spacy.load("lv_roberta_large")

print("lv_roberta_large Pipeline组件:", nlp_xlmr.pipe_names)

Processing ./packages/lv_roberta_large-1.0.0/dist/lv_roberta_large-1.0.0-py3-none-any.whl
lv_roberta_large Pipeline组件: ['transformer', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [12]:
# =============================
# Cell 10B: 通过'tar.gz'文件安装
# =============================
import subprocess
import spacy
import pandas as pd

# 安装 '.tar.gz'
subprocess.run(["pip", "install", "./packages/lv_roberta_large-1.0.0/dist/lv_roberta_large-1.0.0.tar.gz"])
nlp_xlmr = spacy.load("lv_roberta_large")

print("lv_roberta_large Pipeline组件:", nlp_xlmr.pipe_names)

Processing ./packages/lv_roberta_large-1.0.0/dist/lv_roberta_large-1.0.0.tar.gz
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  DEPRECATION: Building 'lv_roberta_large' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'lv_roberta_large'. Discussion can be found at https://github.com/pypa/pip/issues/6334


  Created wheel for lv_roberta_large: filename=lv_roberta_large-1.0.0-py3-none-any.whl size=1839191015 sha256=94430634bd61bd9272658892b2b505768d44308036604caa8f5b76fb8b4fab91
  Stored in directory: /home/jesse/.cache/pip/wheels/6f/60/ad/25525e1dac412aadde9f1cc07d01b8c5e52e241927647194f0
Successfully built lv_roberta_large
  Attempting uninstall: lv_roberta_large
    Found existing installation: lv_roberta_large 1.0.0
    Uninstalling lv_roberta_large-1.0.0:
      Successfully uninstalled lv_roberta_large-1.0.0
lv_roberta_large Pipeline组件: ['transformer', 'tagger', 'morphologizer', 'parser', 'senter', 'lemmatizer']


In [13]:
# ========================================
# Cell 11: 测试模型，显示 Lemma，并显示分句
# ========================================
import spacy
import pandas as pd

nlp = spacy.load("lv_roberta_large")

text = """Rīga ir Latvijas galvaspilsēta un viens no galvenajiem rūpniecības, darījumu, kultūras, sporta un finanšu centriem Baltijas valstīs, kā arī nozīmīga ostas pilsēta. Ar 605 273 iedzīvotājiem (2024. gada dati) tā ir lielākā apdzīvotā vieta Latvijā. Tās robežās dzīvo aptuveni viena trešdaļa, bet Rīgas aglomerācijā — vairāk nekā puse visu Latvijas iedzīvotāju. Pilsētas teritorijas platība ir 307,17 km2. Rīgas vēsturiskais centrs ir iekļauts UNESCO Pasaules kultūras mantojuma sarakstā un ir ievērojams ar jūgendstila arhitektūru, kurai, pēc UNESCO viedokļa, nav līdzīgu pasaulē. Kopš dibināšanas 1201. gadā līdz mūsu dienām Rīga ir Baltijas valstu lielākā pilsēta un viena no ievērojamākajām ostām Baltijas jūras austrumdaļā. Politiski un administratīvi tā ilgu laiku bijusi reģiona politiskais centrs, bet sākot ar 20. gadsimtu — Latvijas Republikas galvaspilsēta."""

doc = nlp(text)

# 先生成 token 表格
rows = []
for sent in doc.sents:
    for token in sent:
        rows.append({
            "Text": token.text,
            "Lemma": token.lemma_,
            "POS": token.pos_,
            "Dependency": token.dep_,
            "Head": token.head.text
        })

df = pd.DataFrame(rows)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
display(df)  # 显示 token 表格

# 再单独显示分句
print("\n✅ 分句结果：")
for i, sent in enumerate(doc.sents, 1):
    print(f"Sentence {i}: {sent.text}")

,Text,Lemma,POS,Dependency,Head
0,Rīga,Rīga,PROPN,nsubj,galvaspilsēta
1,ir,būt,AUX,cop,galvaspilsēta
2,Latvijas,Latvijas,PROPN,nmod,galvaspilsēta
3,galvaspilsēta,galvaspilsēta,NOUN,ROOT,galvaspilsēta
4,un,un,CCONJ,cc,viens
5,viens,viens,NUM,conj,galvaspilsēta
6,no,no,ADP,case,centriem
7,galvenajiem,galvenais,ADJ,amod,centriem
8,rūpniecības,rūpniecība,NOUN,nmod,centriem
9,",",",",PUNCT,punct,darījumu



✅ 分句结果：
Sentence 1: Rīga ir Latvijas galvaspilsēta un viens no galvenajiem rūpniecības, darījumu, kultūras, sporta un finanšu centriem Baltijas valstīs, kā arī nozīmīga ostas pilsēta.
Sentence 2: Ar 605 273 iedzīvotājiem (2024. gada dati) tā ir lielākā apdzīvotā vieta Latvijā.
Sentence 3: Tās robežās dzīvo aptuveni viena trešdaļa, bet Rīgas aglomerācijā — vairāk nekā puse visu Latvijas iedzīvotāju.
Sentence 4: Pilsētas teritorijas platība ir 307,17 km2.
Sentence 5: Rīgas vēsturiskais centrs ir iekļauts UNESCO Pasaules kultūras mantojuma sarakstā un ir ievērojams ar jūgendstila arhitektūru, kurai, pēc UNESCO viedokļa, nav līdzīgu pasaulē.
Sentence 6: Kopš dibināšanas 1201. gadā līdz mūsu dienām Rīga ir Baltijas valstu lielākā pilsēta un viena no ievērojamākajām ostām Baltijas jūras austrumdaļā.
Sentence 7: Politiski un administratīvi tā ilgu laiku bijusi reģiona politiskais centrs, bet sākot ar 20. gadsimtu — Latvijas Republikas galvaspilsēta.


In [14]:
import spacy
import numpy as np

# Load the pipeline
# 加载模型流水线
nlp = spacy.load("lv_roberta_large")

# Example text
# 示例文本
text = """Baltijas jūras nosaukums ir devis nosaukumu baltu valodām un Baltijas valstīm.
Terminu "Baltijas jūra" (Mare Balticum) pirmoreiz lietoja vācu hronists Brēmenes Ādams 11. gadsimtā."""

# Process text
# 处理文本
doc = nlp(text)

# ------------------------
# Tokenization / 分词
# ------------------------
print("Tokens / 分词结果:")
print([token.text for token in doc])

# ------------------------
# Lemmatization / 词形还原
# ------------------------
print("Lemmas / 词形还原结果:")
print([token.lemma_ for token in doc])

# ------------------------
# Part-of-Speech Tagging / 词性标注
# ------------------------
print("POS tags / 词性标注:")
for token in doc:
    print(f"{token.text}: {token.pos_} ({token.tag_})")
# pos_ 是 Universal POS 标签, tag_ 是语言特定 POS 标签

# ------------------------
# Morphological Features / 形态特征
# ------------------------
print("Morphological features / 形态特征:")
for token in doc:
    print(f"{token.text}: {token.morph}")
# 输出形态信息，如格、数、性别、时态等

# ------------------------
# Dependency Parsing / 依存句法分析
# ------------------------
print("Dependency parsing / 依存句法分析:")
for token in doc:
    print(f"{token.text} <--{token.dep_}-- {token.head.text}")
# 输出每个 token 的依存关系及其父节点

# ------------------------
# Sentence Segmentation / 分句
# ------------------------
print("Sentences / 分句结果:")
for sent in doc.sents:
    print(sent.text)

# ------------------------
# 直接访问流水线组件（可选，高级用法）
# ------------------------
print("Pipeline components / 流水线组件:")
print(nlp.pipe_names)

# Tok2Vec (词向量表示)
vectors = np.vstack([token.vector for token in doc])
print("Token vectors shape / Token 向量维度:", vectors.shape)

Tokens / 分词结果:
['Baltijas', 'jūras', 'nosaukums', 'ir', 'devis', 'nosaukumu', 'baltu', 'valodām', 'un', 'Baltijas', 'valstīm', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietoja', 'vācu', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimtā', '.']
Lemmas / 词形还原结果:
['Baltijas', 'jūra', 'nosaukums', 'būt', 'dot', 'nosaukums', 'balts', 'valoda', 'un', 'Baltijas', 'valsts', '.', '\n', 'Terminu', '"', 'Baltijas', 'jūra', '"', '(', 'Mare', 'Balticum', ')', 'pirmoreiz', 'lietot', 'vāci', 'hronists', 'Brēmenes', 'Ādams', '11', '.', 'gadsimts', '.']
POS tags / 词性标注:
Baltijas: PROPN (npfsg4)
jūras: NOUN (ncfsg4)
nosaukums: NOUN (ncmsn1)
ir: AUX (vcnipii30an)
devis: VERB (vmnpdmsnasnpn)
nosaukumu: NOUN (ncmsa1)
baltu: NOUN (ncmpg1)
valodām: NOUN (ncfpd4)
un: CCONJ (cc)
Baltijas: PROPN (npfsg4)
valstīm: NOUN (ncfpd6)
.: PUNCT (zs)

: PUNCT (r0n)
Terminu: NOUN (ncmsa1)
": PUNCT (zq)
Baltijas: PROPN (npfsg4)
jūra: NOUN (ncfsn4)
": PUNCT (zq)
(: P